# Cleaning Bank Marketing Campaign Data

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

### `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

### `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

### `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

## Solution:

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("bank_marketing.csv")
data
#data.info()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,nov,30,334,1,0,nonexistent,94.767,1.028,yes
41184,41184,46,blue-collar,married,professional.course,no,no,nov,6,383,1,0,nonexistent,94.767,1.028,no
41185,41185,56,retired,married,university.degree,no,yes,nov,24,189,2,0,nonexistent,94.767,1.028,no
41186,41186,44,technician,married,professional.course,no,no,nov,17,442,1,0,nonexistent,94.767,1.028,yes


#### `client.csv`

In [2]:
data = pd.read_csv("bank_marketing.csv")

# job
data["job"] = data["job"].str.replace(".", "_")

# education
data["education"] = data["education"].str.replace(".", "_")
data.loc[data["education"] == "unknown", "education"] = np.NaN

# credit_default 
#set(list(data["credit_default"]))
data["credit_default"] = data["credit_default"].map({"no": False,
                                                     "unknown": np.NaN,
                                                     "yes": True})

# mortgage
#set(list(data["mortgage"]))
data["mortgage"] = data["mortgage"].map({"no": False,
                                         "unknown": np.NaN,
                                         "yes": True})


client = data[["client_id", "age", "job", "marital",
               "education", "credit_default", "mortgage"]]

client.to_csv("clients.csv", index = False)

#### `campaign.csv`

In [3]:
data = pd.read_csv("bank_marketing.csv")

# previous_outcome
#set(list(data["previous_outcome"]))
data["previous_outcome"] = data["previous_outcome"].map({"failure": False,
                                                         "nonexistent": np.NaN,
                                                         "success": True})

# campaign_outcome
#set(list(data["campaign_outcome"]))
data["campaign_outcome"] = data["campaign_outcome"].map({"no": False,
                                                         "yes": True})

# last_contact_date
#set(list(data["month"]))
def MM(month):
    corresponding = {'apr': "04", 'aug': "08",'dec': "12", 'jul': "07",
                     'jun': "06", 'mar': "03", 'may': "05", 'nov': "11",
                     'oct': "08", 'sep': "09"}
    return corresponding[month]
data["last_contact_date"] =  "2022-" + data["month"].apply(MM) + "-" + data["day"].astype(str)


campaign = data[["client_id", "number_contacts", "contact_duration",
                 "previous_campaign_contacts", "previous_outcome",
                 "campaign_outcome", "last_contact_date"]]

campaign.to_csv("campaign.csv", index = False)

#### `economics.csv`

In [4]:
economics = data[["client_id", "cons_price_idx",
                  "euribor_three_months",]]

economics.to_csv("economics.csv", index = False)

## Results:

In [5]:
pd.read_csv("clients.csv")

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,NaN,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False
...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,False,True
41184,41184,46,blue-collar,married,professional_course,False,False
41185,41185,56,retired,married,university_degree,False,True
41186,41186,44,technician,married,professional_course,False,False


In [6]:
pd.read_csv("campaign.csv")

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,NaN,False,2022-05-13
1,1,1,149,0,NaN,False,2022-05-19
2,2,1,226,0,NaN,False,2022-05-23
3,3,1,151,0,NaN,False,2022-05-27
4,4,1,307,0,NaN,False,2022-05-3
...,...,...,...,...,...,...,...
41183,41183,1,334,0,NaN,True,2022-11-30
41184,41184,1,383,0,NaN,False,2022-11-6
41185,41185,2,189,0,NaN,False,2022-11-24
41186,41186,1,442,0,NaN,True,2022-11-17


In [7]:
pd.read_csv("economics.csv")

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857
...,...,...,...
41183,41183,94.767,1.028
41184,41184,94.767,1.028
41185,41185,94.767,1.028
41186,41186,94.767,1.028
